In [32]:
import os

#this module aims to test bam file's format via ValidateSamFile (a java picard tool)
#if VSF output ERROR then break and show user the ERROR info for format correction
#otherwise it shall continue the qc process

location="/home/kechanglin/data/new_test.bam" #input a bam's directory
valinfo=os.popen("java -jar /home/kechanglin/picard.jar ValidateSamFile I="+location+" MODE=SUMMARY")

In [33]:
import re

def validateBAM(valinfo):
    for i in valinfo:
        if not re.match('ERROR',str(i))== None:
            print(i)
    

In [34]:
validateBAM(valinfo)

ERROR:INVALID_MAPPING_QUALITY	39161

ERROR:MISSING_PLATFORM_VALUE	1



In [170]:
import pysam
location1="/data/yangxiaoxia/bqsr.bam"
psentity=pysam.AlignmentFile(location1,'rb')


In [119]:
# BAMheader=pysam.view("-H",location1)
a=str(re.findall(r'PL:\w+',BAMheader))[5:-2] #output platform name
print(a)


mgiseq


In [117]:
print(BAMheader)

@HD	VN:1.6	SO:coordinate
@SQ	SN:chr6	LN:171115067
@RG	ID:1	LB:lib1	PL:mgiseq	SM:test	PU:unit1
@PG	ID:bwa	PN:bwa	VN:0.7.15-r1140	CL:bwa mem chr6.fasta V100007471_L03_539_1.clean.fq.gz V100007471_L03_539_2.clean.fq.gz
@PG	ID:MarkDuplicates	VN:Version:4.1.2.0	CL:MarkDuplicates  --INPUT aln-pe.chr6.sort.bam --OUTPUT marked_duplicates.bam --METRICS_FILE marked_dup_metrics.txt  --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --DUPLEX_UMI false --ADD_PG_TAG_TO_READS true --REMOVE_DUPLICATES false --ASSUME_SORTED false --DUPLICATE_SCORING_STRATEGY SUM_OF_BASE_QUALITIES --PROGRAM_RECORD_ID MarkDuplicates --PROGRAM_GROUP_NAME MarkDuplicates --READ_NAME_REGEX <optimized capture of last three ':' separated fields as numeric values> --OPTICAL_DUPLICATE_PIXEL_DISTANCE 100 --MAX_OPTICAL_DUPLICATE_SET_SIZE 300000 --V

In [213]:
BAMheader[67:76]

'PL:mgiseq'

In [224]:
class BAMinput(object):
    """input bam file and check format"""
    
    def __init__(self,directory):
        self.directory=directory
    
    def ValidateBAM(self,crash=False):
        self.valinfo=os.popen("java -jar /home/kechanglin/picard.jar ValidateSamFile I="+self.directory)
        for i in self.valinfo:
            if not re.match('ERROR',str(i))== None:
                print(i)
                self.crash=True
        if crash==True:
            raise ValueError('fatal error in file format!')
        
    
    def gen_identification(self):
        self.treatment=False
        self.BAMheader=pysam.view("-H",self.directory)
        self.platform=str(re.findall(r'PL:\w+',self.BAMheader))[5:-2] #output platform name
        try:
            if plat[str.upper(self.platform)]=='gen3':
                self.treatment=True #treatment arg is for furture gen3 file handling
        except KeyError as e:
            raise KeyError('no equipment info provided, thus unable to tell sequencing technology, it could be a converted file.')
        finally:
            return self.treatment
        
# @staticmethod
def check_md5(filepath, md5):
    """Check File md5."""
    filemd5 = FileMD5(filepath)
    return filemd5.md5check(md5)
        

In [206]:
class FileMD5(object):
    """Generate md5."""

    def __init__(self, filepath):
        """Init class."""
        filepath = os.path.abspath(filepath)
        if not os.path.isfile(filepath):
            raise ValueError("Can not find file %s!" % filepath)
        self.filepath = filepath

    @property
    def md5(self):
        """Get md5 of file."""
        hash_md5 = hashlib.md5()
        with open(self.filepath, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()

    def write_md5(self, outfile):
        """Write md5 of file to outfile."""
        with open(outfile, 'wt') as wt:
            wt.write('%s  %s' % (self.md5, self.filepath))

    def md5check(self, md5):
        """Check md5 file."""
        if self.md5 != md5:
            return False
        else:
            return True

#maybe useless code
def md5sum(md5file):
    """Md5sum -c file."""
    md5, filename = ('',) * 2
    with open(md5file) as wt:
        for line in wt:
            if line.startswith("MD5"):
                filename, md5 = line.split('=')
                filename = filename.replace('MD5(', '').replace(')', '')
            else:
                md5, filename = line.split('  ')[:2]
            md5 = md5.strip()
            filename = filename.strip()
            filemd5 = FileMD5(filename)
            infor = 'succeed' if filemd5.md5check(md5) else 'fail'
            print('{} md5 check: {}'.format(filename, infor))


In [222]:
import logging
import os
import re
import sys
import time
import argparse
import json
import pysam
import hashlib
import subprocess

plat={
    'PACBIO':'gen3',
    'SEQUEL':'gen3',
    'ILLUMINA':'gen2',
    'MGISEQ':'gen2'
}

qualimap_loc="/home/kechanglin/biosoft/qualimap_v2.2.1/qualimap"
qualimap_out="/home/kechanglin/data"

In [226]:

"""Run system command."""
def run_cmd(cmd):
    pipe = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    stdout,stderr = pipe.communicate()
    pipe.wait()
    if pipe.returncode != 0:
        raise ValueError("Failed to run command :%s, error mesages: %s." % (cmd, pipe.stderr.read().decode('utf-8')))
    else:
        return stdout,stderr
        # return pipe.returncode,stdout,stderr
    

In [140]:
testinstance=BAMinput(location)
testinstance.ValidateBAM()
testinstance.gen_identification()

ERROR:INVALID_MAPPING_QUALITY	39161

ERROR:MISSING_PLATFORM_VALUE	1



ValueError: fatal error in file format!

In [225]:
testinstance=BAMinput(location)
testinstance.ValidateBAM()
#testinstance.gen_identification()

ERROR: Read name a8234680, A platform (PL) attribute was not found for read group 

ERROR: Record 1, Read name m54152_170704_111850/4981715/53632_58642, MAPQ should be 0 for unmapped read.

ERROR: Record 2, Read name m54152_170704_111850/5439729/0_3464, MAPQ should be 0 for unmapped read.

ERROR: Record 3, Read name m54152_170704_111850/5571059/0_4789, MAPQ should be 0 for unmapped read.

ERROR: Record 4, Read name m54152_170704_111850/5636569/0_9505, MAPQ should be 0 for unmapped read.

ERROR: Record 5, Read name m54152_170704_111850/5702167/0_22561, MAPQ should be 0 for unmapped read.

ERROR: Record 6, Read name m54152_170704_111850/5833093/0_9037, MAPQ should be 0 for unmapped read.

ERROR: Record 7, Read name m54152_170704_111850/5833093/9075_14919, MAPQ should be 0 for unmapped read.

ERROR: Record 8, Read name m54152_170704_111850/5833192/0_15718, MAPQ should be 0 for unmapped read.

ERROR: Record 9, Read name m54152_170704_111850/5833243/0_26564, MAPQ should be 0 for unmapped re

In [214]:
check_md5(location1,'093dd0fec383a9d9')

False

In [221]:
qc_info=os.popen(qualimap_loc+' bamqc -bam '+location+' -outdir '+qualimap_out+' -outformat PDF:HTML')
print(qc_info)

In [251]:
qc_sub_run=run_cmd(qualimap_loc+' bamqc -bam '+location+' -outdir '+qualimap_out+' -outformat PDF:HTML')

In [248]:
run_list=str(qc_sub_run[0]).split('\\n')

In [250]:
for i in range(len(run_list)):
    print(run_list[i])


b'Java memory size is set to 1200M
Launching application...

QualiMap v.2.2.1
Built on 2016-10-03 18:14

Selected tool: bamqc
Available memory (Mb): 32
Max memory (Mb): 1118
Wed Jul 31 16:07:23 CST 2019\t\tWARNING\tOutput folder already exists, the results will be saved there

Starting bam qc....
Loading sam header...
Wed Jul 31 16:07:23 CST 2019\t\tWARNING\tNon-standard header SortOrder value!
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 0
Chunk of reads size: 1000
Number of threads: 4
Total processed windows:0
Number of reads: 39161
Number of valid reads: 0
Number of correct strand reads:0
Wed Jul 31 16:07:38 CST 2019\t\tWARNING\tSAMRecordParser marked 39161 problematic reads.

Inside of regions...
Num mapped reads: 0
Num mapped first of pair: 0
Num mapped second of pair: 0
Num singletons: 0
Time taken to analyze reads: 14

Overall analysis time: 14
end of bam qc
Computing report...
Writing PDF:HTML report...
HTML report created success

In [255]:
import hashlib
#individual check on md5
def md5(filepath):
    """Get md5 of file."""
    hash_md5 = hashlib.md5()
    with open(filepath, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

md5(location)
    


'2cb38082d6d46d425cb7181665e38147'